In [ ]:
from experiments.exp_3d.preprocessing import *
from losses.Metrics import metric_dice

from lib.visualize import *
from experiments.exp_3d.inference import Pipeline
import matplotlib.pyplot as plt
import pandas as pd 
import json
import csv

import pydicom
import pydicom_seg
import SimpleITK as sitk
from library_dicom.export_segmentation.ExportSegmentation_Writer import ExportSegmentation_Writer

In [ ]:
target_size = (128, 128, 256)
target_spacing = (4.0, 4.0, 4.0)
target_direction = (1,0,0,0,1,0,0,0,1)
model_path = '/media/oncopole/DD 2To/SEGMENTATION/training/20210429_11h17m32/trained_model_20210429_11h17_32.h5'
target_origin = None
from_pp = False

csv_path = '/media/oncopole/DD 2To/SEGMENTATION/inference/inference_dataset_V5.csv'


df = pd.read_csv(csv_path)
dataset = df[['patient_id', 'study', 'study_id', 'ct_img', 'pet_img', 'mask_img']].to_dict('records')
print("")
pipeline = Pipeline(target_size, target_spacing, target_direction, model_path=model_path, target_origin=None, from_pp=False)
print("Pipeline object initialized")

In [ ]:
len(dataset[0:100])

In [ ]:
from library_dicom.dicom_processor.tools.create_mip import *
import time 

start_time = time.time()
directory = '/media/oncopole/DD 2To/SEGMENTATION/inference'
nifti_directory = '/media/oncopole/DD 2To/SEGMENTATION/inference/nifti'
vmin = 0 
vmax = 10 
liste_paths_images = []
for count, img_path in enumerate(dataset[0:100]) : 
    print(count)
    study_uid = img_path['study_id']
    patient_id = img_path['patient_id']
    study = img_path['study']
    pet_img = sitk.ReadImage(img_path['pet_img'], sitk.sitkFloat32)
    pet_spacing = pet_img.GetSpacing()
    pet_size = pet_img.GetSize()
    pet_origin = pet_img.GetOrigin()
    pet_direction = pet_img.GetDirection()
    pet_array = sitk.GetArrayFromImage(pet_img)


    pred_img = pipeline(img_path)
    #save predictions as nifti 
    
    pred_img.SetSpacing(pet_spacing)
    pred_img.SetDirection(pet_direction)
    pred_img.SetOrigin(pet_origin)
    sitk.WriteImage(pred_img, nifti_directory+'/'+study_uid+'_inference_pred.nii')
    img_path['pred_img'] = nifti_directory+'/'+study_uid+'_inference_pred.nii'

    pred_array = sitk.GetArrayFromImage(pred_img)

    #generate pdf with mip inference
    for type_ in range(1,3):
        filename = inference_pet_projection(pet_array, pred_array, study_uid, patient_id, study,  axis=type_, directory=directory , vmin=vmin, vmax=vmax)
        liste_paths_images.append(filename)
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
from library_dicom.dicom_processor.tools.folders import * 

write_json_file(directory, 'liste_images_paths', liste_paths_images)

In [ ]:
generate_inference_pet_projection_pdf(liste_paths_images, directory, pdf_filename='inference')

In [ ]:
directory = '/media/oncopole/DD 2To/SEGMENTATION/inference'
filename = 'prediction_dataset.csv'

with open(os.path.join(directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow([ "patient_id", "study_id", "study", "ct_img", "pet_img", "mask_img", 'pred_img'])
    for serie in dataset: 
        csv_writer.writerow([serie['patient_id'], serie['study_id'], serie['study'], serie['ct_img'], serie['pet_img'], serie['mask_img'], serie['pred_img']])

In [ ]:
#Dicom SEG 

segmentation = pred_img
pet_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome/1.2.250.1.59.453.859.92444169.1516.1370594209.2.2.8'
filename = 'sarcome_dicom_seg_v2.dcm'
directory_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome'
export_objet = ExportSegmentation_Writer(segmentation, segmentation_type='img', serie_path = pet_path, mode='dicomseg')
export_objet.generate_dicom(filename = filename, directory_path = directory_path)

In [ ]:
#DICOM RTSTRUCT 
from library_dicom.export_segmentation.ExportSegmentation_Writer import ExportSegmentation_Writer
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.export_segmentation.tools.rtss_writer_tools import *
from skimage.measure import label

#Clean the mask + label 


segmentation = pred_img
segmentation =  sitk.GetArrayFromImage(segmentation)
segmentation = clean_mask(segmentation)
labelled = label(segmentation)
#generate rtstruct file 
pet_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome/1.2.250.1.59.453.859.92444169.1516.1370594209.2.2.8'
export_objet = ExportSegmentation_Writer(labelled, segmentation_type ='matrix', serie_path = pet_path, mode = 'rtstruct')
filename = 'sarcome_rtstruct_v2.dcm'
directory_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome'
export_objet.generate_dicom(filename, directory_path)
